In [ ]:
# pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 7.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

from collections import Counter

In [5]:
def delete_punctuation_marks_tag(word):
    word = word.replace(",","")
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("!","")
    word = word.replace(":","")
    word = word.replace("?","")
    word = word.replace("~","")
    word = word.replace("\\","")
    word = word.replace("\"","")
    word = word.replace(";","")
    return word.lower()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restaurant_info.csv')
eat_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restaurant_review.csv')


In [11]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,37.554019,127.094133,448,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,37.545355,127.085301,309,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,37.546406,127.075574,63,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,37.564606,127.081460,568,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,37.541762,127.065878,34,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...
...,...,...,...,...,...,...,...,...,...,...
668,화양주막,서울특별시 광진구 아차산로27길 4 1F,일반 주점,25000,1,NaN,37.541787,127.067684,663,곱도리탕.묵사발.미숫가루막걸리 맛있는곳ㅎㅎ곱창이 4줄이나 있어서 29000원가격이 ...
669,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,35000,10,4.2,37.558149,127.080287,180,성수 유명한 맛집에서 이제는 군자 맛집으로너무 가보고싶었던 곳이라 엄마랑 동생이랑 ...
670,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,35000,19,3.5,37.533938,127.066940,64,뚝섬역에서 가까운 글래시 와인바에요 ! 와인바 찾던 중 우연히 발견했는데 가격도 맛...
671,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,50000,4,NaN,37.554370,127.111210,173,제주도에서 하는 빛의 벙커를 서울에서도 한다고 하여워커힐 방문!빛의 시어터 다 보고...


In [12]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,맛이차이나,김성우,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,3,191,876
1,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,5,448,1049
2,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,5,448,674
3,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,5,448,1596
4,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,5,448,2150
...,...,...,...,...,...,...
7125,맛이차이나,조냠냠,"진짜 맛이 차이나는지는 모르겠고 ㅎ깐풍기 소짜에 크림새우 세트 하나에게살스프, 소고...",0,191,2146
7126,맛이차이나,굿걸,전체적으로 맛이 괜찮았지만왜인지 다시 가고 싶지는 않은 느낌?몇 년만에 다시 갔더니...,3,191,815
7127,맛이차이나,민주,기대를 너무 하고 가지 않길 탕수육은 초마가 훨씬 더 맛있었고 칠리새우는 다 먹지도 않음,0,191,1402
7128,맛이차이나,떠뽀끼쥬떼연,어향가지가 맛있는 중국집 런치 세트가 가성비 갑인 중국집 사실 간짜장이 먹고싶어서 ...,3,191,1119


In [13]:
eat_review_by_user = pd.DataFrame(index = eat_review['u_id'].unique(), columns=['reviews'])

for id in tqdm(eat_review['u_id'].unique()):
    review_all = ''
    for review in eat_review.loc[eat_review['u_id']==id,'review']:
        if type(review) != float:
            review_all += review+'. '
    eat_review_by_user.loc[id,'reviews'] = review_all

  0%|          | 0/2532 [00:00<?, ?it/s]

In [14]:
eat_info['tags']=''
eat_review_by_user['tags']=''

In [15]:
eat_info.head()

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews,tags
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,37.554019,127.094133,448,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,37.545355,127.085301,309,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...,
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,37.546406,127.075574,63,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...,
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,37.564606,127.081460,568,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...,
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,37.541762,127.065878,34,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...,


In [16]:
eat_review_by_user.head()

,reviews,tags
876,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,
1049,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,
674,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,
1596,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,
2150,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,


In [17]:
# with open('../data/stop_words.txt','r',-1,'utf-8') as f:
#     stop_words = set(f.read().split('/'))

In [18]:

stopwords = ['맛','더','것','곳','가격','때','밥','진짜','정도','건대','집','여기','음식','좀','처음','수',
             '생각','사람','방문','주문','정말','친구','바로','다른','가게','바','이','조금','또',
             '내','로우','거','하나','메뉴','요리','그','점','가지','별로','식당','한번','추천','삼선','보고','그냥','꽤','보통','나','입','제','직원',
             '다시','말','왜','그릇','무게','꼭','다음','개','느낌','아주','자리','거리','때문','리뷰',
             '중','도','웨이','팅','일단','사진','저','계속','가장','제일','걸','듯','안','건',
             '명','간이','이유','역시','편이','추가','편','재','먹기','거의','시간','기도','테이블','분','마',
             '담','날','근처','손','땐','못','완전','애정','항상','첫','제대로','입맛','이번','먹음',
             '궁합','최고','원래','가면','예전','뭔가','평점','만','한국','다만','기대','배','막','오위','시작',
             '종류','후','냄새','중간','해먹','번','뒤','배달',
             '그램','점점','갑자기','감','살짝','물','콤','색',
             '간','사실','강추','최애','자체','이전','카','골목','음식점','상태','금방','위생','용서','조리','질','요','그때',
             '널','리지','약간','앞','시로','마루','무난','기본','취향','깨','양','줄','수도','개인','매장','식사','두',
             '생','산다','사을','자극','다소','둘','전체','끼','서비스','이이','맛집','아부','얼','비주','오픈','저녁','으;',
             '마끼','대기','운','재료','퀄리티','점심','소스','포장','내부','할머니','토마','식감','동네','일','혼자','요즘','어마','가득','해',
             '저희','볼','인기','뭐','말씀','평일','세','도착','사이즈','일반','밖','간장','후회','인분','손님','달','자주','함',
             '굳이','모든','매우','거기','인생','학교','크기','분위기','절대','주변','무조건','인정','푹','식','옆','주차','위치','존맛','스타일',
             '이모','무엇','밤','새벽','길','기준','알','자꾸','전형','무척','차','베이스','불어','임','위','음','이제','얘기','이해','비','기분',
             '지금','덜','답','북적','신장','개업','끝','방송','위해','법','특유','모습','비교','부분','웨이팅','옥','은','동이','팀','베','부',
             '하의','의사','케이','쿠라','인테리어','마당','대비','인','워낙','쿠니','반','데','마음','개조','미리','단점','하이','','','','',
             '양념','가성','구의역','술집','양도','세트','직접','시징','가족','체인점','안나',
             '이집','저집','적','시','물이','모두','갓','글','수가','집의','합리','부집','준','해도','갈이'
             ]

In [34]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 8.2 MB/s 
     |████████████████████████████████| 453 kB 70.0 MB/s 


In [35]:
import konlpy
Okt = konlpy.tag.Okt()

def get_tags_to_txt(text):
    Okt_morphs = Okt.pos(str(text)) 
    

    Noun_words = []
    for word, pos in Okt_morphs:
        if pos == 'Noun':
            Noun_words.append(word)
    
    unique_Noun_words = set(Noun_words)
    for word in unique_Noun_words:
        if word in stopwords:  #stopwords
            while word in Noun_words: Noun_words.remove(word)  # 최종결과 : Noun_word
    
    if type(text) != str:
        return ''
    tags = Noun_words
    
    
    tags = ','.join(tags)
    
    return tags

In [36]:
for i in tqdm(eat_info.index):
    eat_info.loc[i,'tags'] = get_tags_to_txt(eat_info.loc[i,'reviews'])
    print(get_tags_to_txt(eat_info.loc[i,'reviews']))
    
for i in tqdm(eat_review_by_user.index):
    eat_review_by_user.loc[i,'tags'] = get_tags_to_txt(eat_review_by_user.loc[i,'reviews'])
    print(get_tags_to_txt(eat_review_by_user.loc[i,'reviews']))

  0%|          | 0/673 [00:00<?, ?it/s]

초당,순두부,순두부,향,전혀,아차산,등산,필수,코스,늘,회전,십분,이상,두부,두부,순두부,콩국수,콩국수,콩국,순두부,두부,질감,젓갈,별미,막걸리,한잔,두부,매력,포인트,막걸리,막걸리,반찬,두부,등산,기,유기농,두부,순환,아차산,등산,서울,시내,풍경,눈,하산,할아버지,두부,모,새우,젓갈,고추,두부,젓갈,향,고추,가루,호불호,사이드,순두부,두부,두부,순두부,순두부,양으로,요거트,타,순두부,두유,술,안주,지평막걸리,요구르트,처럼,톡,부랑,조화,콩국수,기회,아차산,순두부,전,경험,등산,두부,부가,일인,순두부,막걸리,두부,두부,백설기,두부,등산,퍼센트,두부,수록,순두부,주시,멸치젓,달달,두부,할아버지,두부,순두부,공기밥,막걸리,새우젓,아차산,겨울,콩국수,이서,두부,순두부,공기,부랑,순두부,저번,공기밥,히트,천,원,주제,무채,순두부,쯤,무채,막걸리,알밤,막걸리,아차산,막걸리,아차산,막걸리,아차산,아차산,막걸리,강,콩,비지,공짜,아차산역,할아버지,두부,아차산,등산,시원,막걸리,잔,전,택시,타고,등산,여부,순두부,두부,두부,와이프,순두부,동의,순두부,공기밥,스윽스윽,젓갈,두부,파시,두부,삭,김치,막걸리,잔,캬,뜻,막걸리,구비,아침,아침,마침,아차산,막걸리,한잔,두부,두부,아침,두부,기계,소리,윙윙,낭만,아차산,등산,막걸리,잔,립스틱,자국,그대로,헹구,고춧가루,잔,두부,비위,상해,지역,외,정보,아침,일찍,순두부,허옇,두부,냄비,떡,당황,했닼,맘,등산,막걸리,그땐,전이,도토리묵,의아,이상,주관,슴,순두부,무생채,액젓,과,명일,두부,순두부,공기밥,요식업,지배,단어,조합,아시,원조,클리셰,할아버지,출동,아차산,아차,두부,할아버지,두부,소개,두부,관심,갱생,조,증말,기회,그동안,아차산,등산,객,등산,막걸리,한잔,한잔,순두부,모양새,뿌리,젓갈,흐트러트리,유아,퇴행,조,콩,달달,순두부,교도소,도입,갱생,경구,투입,다가,두부,교도소,벽,다가,가끔,갱생,마려우,탈옥,숟가락,퍼먹다,죄,범죄,율,대한민국,전혀,두부,만족,순두부,두부,감동,나머지,순두부,젓갈,삭,일시,지은,죄,할렐루야,요약,등산

  0%|          | 0/2532 [00:00<?, ?it/s]

상수,이차,전국구,팔보채,상수,이차,설명,신라,호텔,중식,팔선,경력,셰프,업,호텔,수준,중식,나볼,진진,과,상수,합,정일,꼽는,중식,진진,화상,중식,이차,중국음식,토요일,꽉,지난번,팔보채,공부,탕,대한,호평,게살,볶음밥,먼저,팔보채,양은,구성,전복,해삼,새우,문어,등,외,버섯,채소,해산물,부담,저번,짬뽕,이차,해산물,게살,볶음밥,과,공부,탕,아쉬움,표현,무리,상황,영향,주지,홀,종업원,대학생,응대,또한,상수,이차,진진,과,상수,합정,상권,중식,상수역,중식,홍대,상권,진진,과,중국,꼽,신라,호텔,중식,팔선,경력,셰프,호텔,출신,방장,쉐프,중식,중식,호텔,수준,요미,회,짜장면,소개,중국집,심화,중식,처,분명,존재,이차,꼽는,각기,공부,탕,과,팔보채,를,눈,또한,멘보샤,의,식빵,의,사용,마치,돈까스,전문점,돈,의,사장,빵가루,찾기,제주도,베이커리,계약,신경,디테,일이,더욱,탕수육,소,과,새우,로,구성,짜장면,과,짬뽕,새우,크림,칠리,선택,탕수육,호기롭,찍,형태,탕수육,필요,튀김,옷,형성,바삭,존재,안심,툼,감탄,마치,돈까스,프리미엄,돈까스,접,랄,과일,농축,단맛,크림,새우,마리,새우,튀김,크림소스,누룽지,튀김,크림소스,부담,새우,향,짜장면,단,의도,짬뽕,닭,육수,해물,인상,오징어,결,해산물,잔뜩,공부,탕,꼽는,중식,호텔,수준,중식,최대,장점,접근성,또한,코스,또한,구성,층,주로,홀로,운영,층,룸,운영,만남,장소
초당,순두부,순두부,향,전혀,동치미,향,맛뒤,육수,혼밥,김밥,잔치국수,쫄깃함신맛,익,동치미,역대,최악,어디,서도,보지,스프,카레,어찌,고민,서서,속도,돈코츠,라멘,실패
아차산,등산,필수,코스,늘,회전,십분,이상,두부,두부,순두부,콩국수,콩국수,콩국,순두부,두부,질감,젓갈,별미,막걸리,한잔,육수,고명,커버,오천원,아래쪽,핫도그,깜짝,스테이크,감자,튀김,회,냉면,매콤,달콤,맵,촐,냉면집,면,평냉,뚝뚝,이로,회심,냉면,함흥,냉면,갈매기살,갈매기살,여름,휴가,여름,전화,가세,소바,우동,겨울,시즌,우동,자루,소바,면,튀김,류,선,듬,튀김,해결,면발,쫄깃,밀가루,남,소

In [37]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews,tags
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,37.554019,127.094133,448,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,"초당,순두부,순두부,향,전혀,아차산,등산,필수,코스,늘,회전,십분,이상,두부,두부,..."
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,37.545355,127.085301,309,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...,"서울특별시,광진구,구의,동,서북,면옥,물냉면,과,접시,만두,물냉면,육수,슴슴,소고기..."
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,37.546406,127.075574,63,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...,"핫도그,문,준비,고려,핫도그,특성,아쉬움,소세지,굽기,당선,양은,간식,광진구,투어,..."
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,37.564606,127.081460,568,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...,"온,첫날,짜장면,망플,장안,분식집,사장,대한민국,요식업,친절,장안,떡볶이,튀김,오뎅..."
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,37.541762,127.065878,34,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...,"혼밥,김밥,잔치국수,잔치국수,김밥,잔치국수,다대기,움,김밥,계란,당근,단무지,위주,..."
...,...,...,...,...,...,...,...,...,...,...,...
668,화양주막,서울특별시 광진구 아차산로27길 4 1F,일반 주점,25000,1,NaN,37.541787,127.067684,663,곱도리탕.묵사발.미숫가루막걸리 맛있는곳ㅎㅎ곱창이 4줄이나 있어서 29000원가격이 ...,"곱,도리,탕,묵,사발,미숫가루,막걸리,곱창,곱창,사리,값,곱창,젤,마니쫄,매콤,묵,..."
669,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,35000,10,4.2,37.558149,127.080287,180,성수 유명한 맛집에서 이제는 군자 맛집으로너무 가보고싶었던 곳이라 엄마랑 동생이랑 ...,"성수,군자,엄마,동생,와인,가야,양파,스프,꼬기,더욱,군자역,거주,인구,군자,집들이..."
670,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,35000,19,3.5,37.533938,127.066940,64,뚝섬역에서 가까운 글래시 와인바에요 ! 와인바 찾던 중 우연히 발견했는데 가격도 맛...,"뚝섬역,래시,와인,와인,발견,세번,파스타,시중,사장,면,관건,파스타,와인,와인,발견..."
671,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,50000,4,NaN,37.554370,127.111210,173,제주도에서 하는 빛의 벙커를 서울에서도 한다고 하여워커힐 방문!빛의 시어터 다 보고...,"제주도,빛,벙커,서울,워커힐,빛,시어터,워커힐,뷰,말모,층,쪽,보틀,일정,금액,이상..."


In [38]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_info.csv')

In [39]:
# eat_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/eat_info.csv')

In [40]:
eat_review_by_user

,reviews,tags
876,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,"상수,이차,전국구,팔보채,상수,이차,설명,신라,호텔,중식,팔선,경력,셰프,업,호텔,..."
1049,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,"초당,순두부,순두부,향,전혀,동치미,향,맛뒤,육수,혼밥,김밥,잔치국수,쫄깃함신맛,익..."
674,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,"아차산,등산,필수,코스,늘,회전,십분,이상,두부,두부,순두부,콩국수,콩국수,콩국,순..."
1596,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,"두부,매력,포인트,막걸리,막걸리,반찬,두부,갑,스시,마카,만칠천원,입성,마카,진행,..."
2150,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,"등산,기,유기농,두부,순환,아차산,등산,서울,시내,풍경,눈,하산,할아버지,두부,모,..."
...,...,...
434,분위기는 괜찮아요 저녁에 시끄럽긴 해도 테이블마다 열린 룸? 분리돼있어요 근디 굴튀...,"룸,분리,디,굴,튀김,분식집,돈까스,피스,화가,남"
2485,- 쉐프코스(45000원)- 양장피 >게살스프 >아스파라거스&전복 >어향가지 >크림...,"쉐프,코스,양장,피,게살,스프,아스파라거스,전복,어향가,크림,새우,탕수육,공부,탕,..."
1621,중식 천재의 데뷔작.맛이차이나는 최근 몇 년간 한국에서 가장 뜨거운 중식당이다.신라...,"중식,천재,데뷔,작,최근,몇,년,중식,신라,호텔,출신,조승희,셰프,독립,업,이차,로..."
333,"의심의 여지 없는 최고의 중국집 중 한 곳! 이라고 표현한다면, 더 유명하고 비싼 ...","의심,여지,중국집,표현,중국,집들이,례,고려,크게,표현,크림,새우,탕수육,보,샤를,..."


In [41]:
eat_review_by_user.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restaurant_review_by_user.csv')

In [42]:
# pip install gensim

In [43]:
# w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
import gensim
w2v = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/ko/ko.bin')

In [44]:
All_tags = set()
for tags in eat_info['tags']:
    All_tags.update(set(tags.split(',')))

In [45]:
new_df = pd.DataFrame()
for word in All_tags:
    try:
        f_vec = w2v.wv.get_vector(word)
        new_df[word] = f_vec
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


In [46]:
new_df

,투자,신경,검색,증언,번의,턱,이서,해운대,버그,가드,...,피부,귀국,촉,휴학,로맨틱,마이너스,매입,사투,아라비아,태사
0,1.446712,-0.281302,0.713723,-0.607761,-1.181960,0.428174,0.958877,0.073695,-1.450894,0.025473,...,1.263762,-1.015237,3.168798,0.060790,-0.331345,-0.143984,0.186212,0.130638,3.475909,0.390970
1,0.572291,0.000811,-2.086323,-0.119170,-1.762698,-0.648297,1.093832,0.996032,0.082478,-0.575517,...,1.345470,0.157969,0.500233,-0.337060,-0.445934,-0.513628,1.717301,-0.214343,-0.993421,0.102958
2,0.160693,-0.078663,-2.727946,0.089713,-0.522883,-0.456992,0.471890,0.579863,0.054083,-0.103052,...,0.033846,0.540198,-0.611305,-0.058376,0.827557,0.108949,0.145173,-0.164371,-0.917228,-0.426914
3,-0.869732,0.664616,0.979134,-1.061416,2.242543,-0.319901,-0.427544,0.076136,2.614210,0.622591,...,-1.043428,-1.499779,0.380542,-0.451890,-0.095260,-0.027667,-0.751541,-0.296177,1.179350,0.439018
4,-0.094220,-0.785873,-3.235434,0.534593,1.404258,0.478581,0.282002,-0.559880,-2.576686,-0.125970,...,-0.838996,0.836642,-0.393055,0.039734,0.059651,-0.372254,-0.236757,-0.363933,1.877349,0.002834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-0.355125,2.064697,-0.103804,0.812840,-0.438374,0.528740,-0.006648,0.927235,-0.050930,-0.461471,...,1.452913,0.661805,1.491744,0.535567,-0.450024,-0.138204,-1.881732,-0.379543,-0.430563,0.642394
196,0.497532,0.847503,0.825118,-2.244581,-0.310464,1.199491,-0.962142,-0.540935,0.232957,0.270277,...,1.938769,0.751309,-0.259648,0.437379,0.235605,0.268911,0.447610,0.419704,-0.131453,-0.139273
197,0.007078,1.577203,1.198191,-0.392267,0.238570,0.180914,-0.956296,-0.600362,0.282377,0.551533,...,2.079481,-1.172090,1.575659,-0.246923,0.178845,0.232575,-1.615865,0.411333,1.509020,-0.776184
198,-0.532298,-0.012458,-0.592791,-0.531534,-1.303941,-0.690563,0.370003,-0.473880,0.221775,0.097609,...,0.397079,0.496690,2.269479,0.541180,0.258073,-0.070260,-0.139425,0.040677,-0.469811,1.024528


In [47]:
x = new_df.T
cluster_count=120
model = KMeans(n_clusters=cluster_count)
model.fit(x)
model.predict(x)
Y=x.copy()
Y['kmeans_id'] = model.predict(x)

In [48]:
km = list(Y['kmeans_id'])
max_count = 0
max_count_id = ''
for i in range (0, cluster_count):
    if km.count(i) > max_count:
        max_count = km.count(i);
        max_count_id = i;
    print(i,': ', km.count(i))
max_count_id

0 :  45
1 :  80
2 :  13
3 :  32
4 :  33
5 :  17
6 :  36
7 :  78
8 :  1
9 :  4
10 :  1
11 :  24
12 :  94
13 :  43
14 :  16
15 :  34
16 :  4
17 :  20
18 :  28
19 :  32
20 :  55
21 :  297
22 :  28
23 :  17
24 :  8
25 :  25
26 :  6
27 :  2
28 :  37
29 :  17
30 :  74
31 :  14
32 :  141
33 :  21
34 :  86
35 :  26
36 :  57
37 :  26
38 :  36
39 :  78
40 :  3
41 :  4
42 :  139
43 :  34
44 :  78
45 :  39
46 :  39
47 :  2
48 :  4
49 :  9
50 :  39
51 :  46
52 :  511
53 :  26
54 :  23
55 :  27
56 :  11
57 :  16
58 :  41
59 :  20
60 :  3
61 :  34
62 :  17
63 :  29
64 :  786
65 :  39
66 :  37
67 :  30
68 :  38
69 :  36
70 :  54
71 :  13
72 :  33
73 :  71
74 :  38
75 :  55
76 :  15
77 :  50
78 :  52
79 :  2
80 :  222
81 :  35
82 :  44
83 :  5
84 :  67
85 :  39
86 :  57
87 :  270
88 :  30
89 :  143
90 :  2
91 :  11
92 :  30
93 :  3
94 :  68
95 :  24
96 :  29
97 :  1
98 :  52
99 :  54
100 :  20
101 :  104
102 :  4
103 :  27
104 :  1
105 :  60
106 :  2
107 :  1
108 :  161
109 :  22
110 :  3
111 :  1
112 

64

In [49]:
km_df = pd.DataFrame()
km_df['group_'+str(max_count_id)] = pd.Series(list(Y[Y['kmeans_id'] == max_count_id].index))
for i in range(0, cluster_count):
    col_name = 'group_'+str(i)
    words = list(Y[Y['kmeans_id'] == i].index)
    km_df[col_name] = pd.Series(words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [50]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
km_df.fillna(0)

Output hidden; open in https://colab.research.google.com to view.

In [51]:
# remove groups we don't need
# rm_groups = [17, 26, 50, 63]
# for i in rm_groups:
#     group_name = 'group_'+str(i);
#     stop_words.update(set(km_df[group_name].dropna()))
# with open('../data/stop_words.txt','w',-1,'utf-8') as f:
#     f.write('/'.join(stop_words))

In [ ]:
del eat_info['Unnamed: 0']
eat_info.head()

KeyError: ignored

In [52]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_info_tags.csv')

In [53]:
res_matrix = pd.DataFrame(index=eat_info.index,columns=list(km_df.columns))
user_matrix = pd.DataFrame(index=eat_review_by_user.index,columns=list(km_df.columns))
res_matrix = res_matrix.fillna(0)
user_matrix = user_matrix.fillna(0)

In [54]:
res_matrix.head()

,group_64,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
user_matrix.head()

,group_64,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
876,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
674,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
for id in tqdm(res_matrix.index):
    tags = Counter(eat_info.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                res_matrix.loc[id,'group_'+str(num)] += tags[tag]

  0%|          | 0/673 [00:00<?, ?it/s]

In [57]:
res_matrix.head()

,group_64,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
0,274,19,14,1,8,4,6,16,9,0,0,3,23,4,1,2,4,2,3,6,9,6,61,4,5,1,12,2,0,11,6,7,6,21,2,11,1,11,6,12,9,0,4,30,11,67,18,2,2,3,2,6,8,76,7,0,5,0,0,4,2,0,15,8,7,18,22,10,3,15,13,0,22,22,6,26,3,30,18,2,30,20,11,0,20,20,5,372,17,28,0,0,0,2,13,8,11,1,7,8,1,14,2,11,0,5,0,2,42,1,0,0,41,28,31,7,0,4,4,15
1,77,6,11,2,5,9,4,8,6,0,0,1,78,5,13,4,2,0,8,7,5,2,46,0,2,0,8,0,0,9,6,5,4,20,2,18,0,9,8,12,24,1,1,31,9,175,11,4,3,1,3,7,2,44,9,2,0,2,1,2,6,0,3,12,11,12,17,7,89,6,8,2,2,9,12,26,3,13,7,0,11,3,7,0,10,19,8,284,17,28,0,0,0,1,17,6,4,0,2,14,1,24,0,3,0,10,0,3,27,0,0,0,53,14,22,9,0,8,2,5
2,26,7,2,1,2,1,0,1,6,0,1,0,6,17,5,0,1,0,1,0,0,1,7,1,2,0,1,0,0,0,1,8,1,5,1,8,0,1,1,3,6,0,0,3,1,89,0,2,0,0,1,1,1,15,1,0,0,0,1,0,0,0,2,1,1,4,1,1,0,5,0,1,0,6,1,1,1,2,4,0,11,6,3,0,13,6,1,20,5,3,0,0,0,0,8,4,3,0,0,0,0,5,0,2,0,6,0,1,8,0,0,0,7,13,3,5,0,0,0,1
3,23,2,1,1,0,2,1,15,3,0,0,0,2,2,2,2,0,0,1,0,0,2,13,3,1,0,0,0,1,3,0,1,2,10,0,9,0,6,6,2,2,2,2,2,3,93,4,3,0,0,0,15,4,19,3,0,0,0,0,6,1,0,3,1,3,2,8,2,1,5,3,2,5,5,2,8,0,11,7,1,6,2,7,0,7,6,0,45,1,7,0,1,0,0,9,5,0,0,0,6,0,4,0,0,0,3,0,0,9,14,0,0,5,6,3,5,0,0,6,2
4,4,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,1,0,0,2,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,4,0,4,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,2,0,0,2,0,1,0,1,0,2,1,1,43,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,1,0,0,0,5,0,1,0,1,0,0


In [58]:
for id in tqdm(user_matrix.index):
    tags = Counter(eat_review_by_user.loc[id,'tags'].split(','))
    for tag in tags.keys():
        for num in range(0,cluster_count):
            if tag in set(km_df['group_'+str(num)]):
                user_matrix.loc[id,'group_'+str(num)] += tags[tag]


  0%|          | 0/2532 [00:00<?, ?it/s]

In [59]:
user_matrix.head()

,group_64,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
876,12,2,0,0,1,4,0,10,1,0,0,0,0,1,2,4,1,0,1,0,0,0,4,0,0,0,0,0,0,1,6,0,2,2,0,0,0,2,3,8,0,0,0,1,1,11,0,0,0,0,0,2,1,4,1,0,0,0,0,7,1,0,0,1,1,0,3,0,1,2,2,0,1,3,0,1,0,0,0,1,5,0,2,0,1,0,1,8,0,23,0,0,0,0,2,1,2,0,0,0,0,0,1,0,0,0,0,1,3,0,0,0,6,2,1,2,0,0,0,0
1049,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,2,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,2,0,0,0,2,0,0,0,0,0,0,0
674,7,0,0,0,1,0,1,2,0,0,0,0,3,1,1,0,0,0,1,0,0,0,2,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,3,1,0,0,1,2,0,18,1,0,0,1,0,0,0,5,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,5,0,0,0,0,3,1,1,0,2,0,1,21,1,0,0,2,0,0,0,3,1,0,0,0,0,5,0,0,0,0,0,0,3,0,0,0,2,3,0,2,0,0,0,0
1596,7,1,1,0,0,0,0,5,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,2,0,1,2,0,0,0,1,0,9,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,1,3,1,0,0,0,0,2,0,4,0,0,0,1,0,0,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,1,1,1,0,1,0,0
2150,3,0,0,0,0,0,0,1,0,0,0,0,2,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0


In [60]:
user_matrix.reset_index()

Output hidden; open in https://colab.research.google.com to view.

In [61]:
user_matrix.rename(columns ={'index':'u_id'})

Output hidden; open in https://colab.research.google.com to view.

In [62]:
user_matrix.head()

,group_64,group_0,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25,group_26,group_27,group_28,group_29,group_30,group_31,group_32,group_33,group_34,group_35,group_36,group_37,group_38,group_39,group_40,group_41,group_42,group_43,group_44,group_45,group_46,group_47,group_48,group_49,group_50,group_51,group_52,group_53,group_54,group_55,group_56,group_57,group_58,group_59,group_60,group_61,group_62,group_63,group_65,group_66,group_67,group_68,group_69,group_70,group_71,group_72,group_73,group_74,group_75,group_76,group_77,group_78,group_79,group_80,group_81,group_82,group_83,group_84,group_85,group_86,group_87,group_88,group_89,group_90,group_91,group_92,group_93,group_94,group_95,group_96,group_97,group_98,group_99,group_100,group_101,group_102,group_103,group_104,group_105,group_106,group_107,group_108,group_109,group_110,group_111,group_112,group_113,group_114,group_115,group_116,group_117,group_118,group_119
876,12,2,0,0,1,4,0,10,1,0,0,0,0,1,2,4,1,0,1,0,0,0,4,0,0,0,0,0,0,1,6,0,2,2,0,0,0,2,3,8,0,0,0,1,1,11,0,0,0,0,0,2,1,4,1,0,0,0,0,7,1,0,0,1,1,0,3,0,1,2,2,0,1,3,0,1,0,0,0,1,5,0,2,0,1,0,1,8,0,23,0,0,0,0,2,1,2,0,0,0,0,0,1,0,0,0,0,1,3,0,0,0,6,2,1,2,0,0,0,0
1049,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,2,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,2,0,0,0,2,0,0,0,0,0,0,0
674,7,0,0,0,1,0,1,2,0,0,0,0,3,1,1,0,0,0,1,0,0,0,2,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,3,1,0,0,1,2,0,18,1,0,0,1,0,0,0,5,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,5,0,0,0,0,3,1,1,0,2,0,1,21,1,0,0,2,0,0,0,3,1,0,0,0,0,5,0,0,0,0,0,0,3,0,0,0,2,3,0,2,0,0,0,0
1596,7,1,1,0,0,0,0,5,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,2,0,1,2,0,0,0,1,0,9,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,1,3,1,0,0,0,0,2,0,4,0,0,0,1,0,0,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,1,1,1,0,1,0,0
2150,3,0,0,0,0,0,0,1,0,0,0,0,2,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0


In [67]:
user_matrix.shape

(2532, 120)

In [64]:
user_matrix.set_index('u_id', drop=True, inplace=True)


KeyError: ignored

In [ ]:
km_df.count()

In [65]:
eat_review_score = pd.DataFrame(eat_review, columns=['u_id','p_id','rating'])
eat_review_score.sort_values(by=['u_id'], axis=0,inplace=True)
eat_review_score.reset_index(drop = True, inplace=True)
eat_review_score.head()

,u_id,p_id,rating
0,0,498,0
1,0,332,5
2,0,139,0
3,0,254,0
4,0,199,3


In [66]:
res_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_res_matrix.csv')
user_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_user_matrix.csv')
eat_review_score.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_score_board.csv',index=False)
km_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_eat_cluster.csv',index=False)
km_df.count().to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restuarant_group_count.csv')